In [39]:
import yfinance as yf
import pandas as pd

# Define stock indices to collect
indices = {"S&P 500": "SPY", "NASDAQ": "QQQ"}

# Define time range
start_date = "1900-01-01"
end_date = "2024-12-31"

# Download and save data
for name, ticker in indices.items():
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Flatten the MultiIndex into single-level column names
    data.columns = ['_'.join(col).strip() for col in data.columns.values]
    data.reset_index(inplace=True)
    data.rename(columns={'Close_SPY': 'Close', 'High_SPY': 'High', 'Low_SPY': 'Low', 
                   'Open_SPY': 'Open', 'Volume_SPY': 'Volume'}, inplace=True)
    data.rename(columns={'Close_QQQ': 'Close', 'High_QQQ': 'High', 'Low_QQQ': 'Low', 
                   'Open_QQQ': 'Open', 'Volume_QQQ': 'Volume'}, inplace=True)
    
    # Convert the 'Date' column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Verify the change
    print(data.dtypes)
    print(data.head())
    print(data.columns)
    data.to_csv(f"../data/raw/{name.replace(' ', '_').lower()}.csv", index=True)
    print(f"{name} data saved successfully.")

from fredapi import Fred
import pandas as pd

fred = Fred(api_key='e916710d165717e6348556cdce8111f3')

# Define economic indicators to collect
indicators = {
    "GDP Growth": "GDP",
    "Unemployment Rate": "UNRATE",
    "CPI": "CPIAUCSL",
    "PPI": "PPIACO",
    "M2 Money Supply": "M2",
    "Interest Rate": "DGS10"
}

# Fetch and save data
for name, series_id in indicators.items():
    data = fred.get_series(series_id)
    data = data.reset_index()
    data.columns = ['Date', name]
    data.to_csv(f"../data/raw/{name.replace(' ', '_').lower()}.csv", index=False)
    print(f"{name} data saved successfully.")

def label_market(data, column='Close'):
    """
    Labels market as Bull (1), Bear (-1), or Neutral (0) based on 10% rules.

    Args:
        data (pd.DataFrame): Stock price data with a 'Close' column.
        column (str): The column to analyze for market trends.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Market_Label' column.
    """
    data['Peak'] = data[column].cummax()
    data['Trough'] = data[column].cummin()
    
    data['Bull_Rule'] = (data[column] >= data['Trough'] * 1.10).astype(int)
    data['Bear_Rule'] = (data[column] <= data['Peak'] * 0.90).astype(int)

    # Assign labels
    data['Market_Label'] = 0  # Neutral
    data.loc[data['Bull_Rule'] == 1, 'Market_Label'] = 1  # Bull
    data.loc[data['Bear_Rule'] == 1, 'Market_Label'] = -1  # Bear

    return data.drop(['Peak', 'Trough', 'Bull_Rule', 'Bear_Rule'], axis=1)

# label S&P500 and NASDAQ:
sp500 = pd.read_csv("../data/raw/s&p_500.csv", parse_dates=['Date'], index_col='Date')
labeled_data = label_market(sp500)
labeled_data.to_csv("../data/processed/labeled_sp500.csv")
print(labeled_data.head())
print(labeled_data.tail())

qqq = pd.read_csv("../data/raw/s&p_500.csv", parse_dates=['Date'], index_col='Date')
labeled_data = label_market(qqq)
labeled_data.to_csv("../data/processed/labeled_qqq.csv")
print(labeled_data.head())
print(labeled_data.tail())

# Load labeled market data
labeled_sp500 = pd.read_csv("../data/processed/labeled_sp500.csv", parse_dates=['Date'])

# Load economic data
gdp = pd.read_csv("../data/raw/gdp_growth.csv", parse_dates=['Date'])
cpi = pd.read_csv("../data/raw/cpi.csv", parse_dates=['Date'])
interest_rate = pd.read_csv("../data/raw/interest_rate.csv", parse_dates=['Date'])
m2 = pd.read_csv("../data/raw/m2_money_supply.csv", parse_dates=['Date'])
ppi = pd.read_csv("../data/raw/ppi.csv", parse_dates=['Date'])
unemployment_rate = pd.read_csv("../data/raw/unemployment_rate.csv", parse_dates=['Date'])

# Merge datasets on Date: S&P500
merged_data_sp500 = labeled_sp500.merge(gdp, on='Date', how='left')
merged_data_sp500 = merged_data.merge(cpi, on='Date', how='left')
merged_data_sp500 = merged_data.merge(interest_rate, on='Date', how='left')
merged_data_sp500 = merged_data.merge(m2, on='Date', how='left')
merged_data_sp500 = merged_data.merge(ppi, on='Date', how='left')
merged_data_sp500 = merged_data.merge(unemployment_rate, on='Date', how='left')

# Save final combined dataset: S&P500
merged_data_sp500.to_csv("../data/processed/combined_data_sp500.csv", index=False)
print("Combined dataset saved successfully.")

# Merge datasets on Date: NASDAQ
merged_data_nasdaq = labeled_sp500.merge(gdp, on='Date', how='left')
merged_data_nasdaq = merged_data.merge(cpi, on='Date', how='left')
merged_data_nasdaq = merged_data.merge(interest_rate, on='Date', how='left')
merged_data_nasdaq = merged_data.merge(m2, on='Date', how='left')
merged_data_nasdaq = merged_data.merge(ppi, on='Date', how='left')
merged_data_nasdaq = merged_data.merge(unemployment_rate, on='Date', how='left')

# Save final combined dataset: NASDAQ
merged_data_nasdaq.to_csv("../data/processed/combined_data_nasdaq.csv", index=False)
print("Combined dataset saved successfully.")




[*********************100%***********************]  1 of 1 completed


Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object
        Date      Close       High        Low       Open   Volume
0 1993-01-29  24.526083  24.543527  24.421420  24.543527  1003200
1 1993-02-01  24.700514  24.700514  24.543519  24.543519   480500
2 1993-02-02  24.752844  24.770288  24.630737  24.683068   201300
3 1993-02-03  25.014505  25.031949  24.770291  24.787735   529400
4 1993-02-04  25.119173  25.171505  24.822627  25.101729   531500
Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
S&P 500 data saved successfully.


[*********************100%***********************]  1 of 1 completed


Date      datetime64[ns]
Close            float64
High             float64
Low              float64
Open             float64
Volume             int64
dtype: object
        Date      Close       High        Low       Open   Volume
0 1999-03-10  43.417294  43.497007  42.753014  43.470436  5232000
1 1999-03-11  43.629837  43.988548  42.779560  43.736122  9688600
2 1999-03-12  42.566990  43.496980  42.221565  43.470409  8743600
3 1999-03-15  43.789272  43.842415  42.434143  42.885853  6369000
4 1999-03-16  44.161278  44.347276  43.496999  43.975280  4905800
Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
NASDAQ data saved successfully.
GDP Growth data saved successfully.
Unemployment Rate data saved successfully.
CPI data saved successfully.
PPI data saved successfully.
M2 Money Supply data saved successfully.
Interest Rate data saved successfully.
            Unnamed: 0      Close       High        Low       Open   Volume  \
Date                                  